# モデル：酵母の解糖系のモデル
- B. Teusink, J. Passarge, C. A. Reijenga, E. Esgalhado, C. C. van der Weijden, M. Schepper, M. C. Walsh, B. M. Bakker, K. van Dam, H. V. Westerhoff, and J. L. Snoep. Can yeast glycolysis be understood in terms of in vitro kinetics of the constituent enzymes? Testing biochemistry. _Eur J Biochem._, **267**(17):5313-5329, 2000.

- [PubMed](https://www.ncbi.nlm.nih.gov/pubmed/10951190)  医学生物学論文のデータベース（米国立医学図書館）
- [BioModels](https://www.ebi.ac.uk/biomodels-main/BIOMD0000000064) 数理生命科学モデルのデータベース（欧州バイオインフォマティクス研究所）
  - [Models of the month 記事](https://www.ebi.ac.uk/biomodels-main/static-pages.do?page=ModelMonth%2F2007-01) BioModelsが毎月１つのモデルを取りあげて解説する記事

## モデル構造の概略図

![モデル概要](./Teusink_model.png)

## モデルに含まれる代謝経路

### グルコース輸送
- 細胞外から細胞内へのグルコースの輸送

### 解糖系

## モデルの辺縁（モデルという世界の果て）
- モデルは **有限** であるため、**閉鎖系** にならざるをえない。
- ホンモノの生命システムは **開放系** だが、モデルは **閉鎖系** 。モデルの **辺縁** （モデル世界の果て）にある要素は、本当はモデルの **外側** につながっていて、その影響を受けている。
- モデルでは、それら **辺縁** に位置する要素を固定する場合が多い。

### このモデルの辺縁（ボーダー）
- 細胞外グルコース（Glc_out）
- グリコーゲン（Glycogen）
- トレハロース（Trehalose）
- グリセロール（Glycerol）
- コハク酸（Succinate）
- エタノール（Ethanol）

#### 辺縁の物質の扱い
- これらの物質はシミュレーション中、変化することがない。
  - ホンモノの生命システムで、この **外側** にある要素によって打ち消され、安定していると仮定されている。

# E-Cell3のモデル要素（`Entity`）

## System
- モデル内の **場所**
- `System` は、フォルダの階層構造のように入れ子構造にできる。
- `System` の相対的な場所をフォルダの階層構造のようにスラッシュ（`/`）区切りで表記したものを `SystemPath` という。
  - `SystemPath` の例（このモデル内の `System`）
    - `/` ルートシステム。必ず存在する。このモデルでは細胞外を表す。
    - `/cytosol` 細胞質

## Variable
- モデル内の **変数**。多くの場合は **物質の量** をあらわす。
- 特定のひとつの `System` に所属している。
  - 物質の移動を表現する場合は、`System` ごとに同じ物質を設定してそれらの増減で表現する。
    - このモデルでは、細胞内と細胞外のグルコースを設定して、細胞膜を介した輸送を表現している。

## Process
- モデル内で起こる **できごと** を表す。`Variable` 間の相互作用を表現する。
- 特定のひとつの `System` に所属している。
- 関連する `Variable` と接続している。

## Stepper
- モデル内の時間経過に沿って **変化が起こるタイミング** を司る。
- 各 `Process` は、ひとつの `Stepper` と接続している。
- `Stepper` がステップすると、接続する `Process` が **発火** し、接続した `Variable` に変化を起こす。


# `Entity` の属性（`Property`）

- `Entity` はさまざまな属性（`Property`）を持つ。

## `Variable` の代表的な属性
- `Value` 分子の数。
- `MolarConc` モル濃度（モル/リットル）
- `NumberConc` 分子数濃度（分子の数/リットル）
- `Velocity` 変化速度。単位時間あたりの増減量。

## `Process` の代表的な属性
- `Activity` 反応速度。単位時間あたりの変化量。
- `MolarActivity` 反応速度をモル濃度（モル/リットル）に換算した値。


# FullID と FullPN
- モデル中の `Entity` や `Property` を正確に指示するための表記法
- コロン（`:`）をデリミタ（区切り文字）に使う。

## FullID (Full IDentifier)
- 記法　**`Entityの種類:SystemPath:Entityの名前`** （スペースを挿入してはならない）
- 例 `Variable:/cytosol:PYR`

## FullPN (Full Property Name)
- 記法　**`Entityの種類:SystemPath:Entityの名前:Propertyの名前`** （スペースを挿入してはならない）
- 例 `Variable:/cytosol:PYR:NumberConc`


# E-Cell3を使うための準備


In [ ]:
import imp
try:
    imp.find_module("ecell")
except ImportError:
    import sys
    sys.path.append("/opt/local/lib/python2.7/site-packages")
    sys.path.append("/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages")

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

## %install_ext https://raw.githubusercontent.com/naito/ecell3-ipython/master/extensions/ecell3.py
## %load_ext ecell3

In [ ]:
import biosim_course

# モデル

- EM形式で記述されたモデル。

In [ ]:
EM = '''
Stepper ODEStepper( Default ){}

Stepper PassiveStepper( PSV ) {}

System System( / )
{
    StepperID    Default;

    Variable Variable( SIZE )
    {
        Value    1.0;
        Fixed    1;
    }
    
    Variable Variable( GLCo )
    {
        Name    "Extracellular Glucose";
        Value    50.0;
        Fixed    1;
    }
    
    
}

System System( /cytosol )
{
    StepperID    Default;

    Variable Variable( SIZE )
    {
        Value    1.0;
        Fixed    1;
    }
    
    Variable Variable( GLCi )
    {
        Name    "Glucose in Cytosol";
        Value    0.0987586913354;
        Fixed    0;
    }
    
    Variable Variable( G6P )
    {
        Name    "Glucose 6 Phosphate";
        Value    1.03324562462;
        Fixed    0;
    }
    
    Variable Variable( F6P )
    {
        Name    "Fructose 6 Phosphate";
        Value    0.112812816403;
        Fixed    0;
    }
    
    Variable Variable( F16P )
    {
        Name    "Fructose-1,6 bisphosphate";
        Value    0.601907643133;
        Fixed    0;
    }
    
    Variable Variable( DHAP )
    {
        Name    "Triose-phosphate";
        Value    0.777523539172;
        Fixed    0;
    }
    
    Variable Variable( BPG )
    {
        Name    "1,3-bisphosphoglycerate";
        Value    0.000329573895017;
        Fixed    0;
    }
    
    Variable Variable( P3G )
    {
        Name    "3-phosphoglycerate";
        Value    0.356484042769;
        Fixed    0;
    }
    
    Variable Variable( P2G )
    {
        Name    "2-phosphoglycerate";
        Value    0.044843711804;
        Fixed    0;
    }
    
    Variable Variable( PEP )
    {
        Name    Phosphoenolpyruvate;
        Value    0.0736168441039;
        Fixed    0;
    }
    
    Variable Variable( PYR )
    {
        Name    Pyruvate;
        Value    8.52315268169;
        Fixed    0;
    }
    
    Variable Variable( ACE )
    {
        Name    Acetaldehyde;
        Value    0.170114457481;
        Fixed    0;
    }
    
    Variable Variable( P )
    {
        Name    "High energy phosphates";
        Value    6.30888166041;
        Fixed    0;
    }
    
    Variable Variable( NAD )
    {
        Name    NAD;
        Value    1.54555976839;
        Fixed    0;
    }
    
    Variable Variable( NADH )
    {
        Name    NADH;
        Value    0.0444402316101;
        Fixed    0;
    }
    
    Variable Variable( Glycogen )
    {
        Name    Glycogen;
        Value    0.0;
        Fixed    1;
    }
    
    Variable Variable( Trehalose )
    {
        Name    Trehalose;
        Value    0.0;
        Fixed    1;
    }
    
    Variable Variable( CO2 )
    {
        Name    CO2;
        Value    1.0;
        Fixed    1;
    }
    
    Variable Variable( Succinate )
    {
        Name    Succinate;
        Value    0.0;
        Fixed    1;
    }
    
    Variable Variable( Ethanol )
    {
        Name    Ethanol;
        Value    50.0;
        Fixed    1;
    }
    
    Variable Variable( Glycerol )
    {
        Name    Glycerol;
        Value    0.15;
        Fixed    1;
    }
    
    Variable Variable( ATP )
    {
        Name    "ATP concentration";
        Value    2.50839064115;
        Fixed    0;
    }
    
    Variable Variable( ADP )
    {
        Name    "ADP concentration";
        Value    1.29210037812;
        Fixed    0;
    }
    
    Variable Variable( AMP )
    {
        Name    "AMP concentration";
        Value    0.299508980733;
        Fixed    0;
    }
    
    Variable Variable( SUM_P )
    {
        Name    "sum of AXP conc";
        Value    4.1;
        Fixed    1;
    }
    
    Variable Variable( F26BP )
    {
        Name    "F2,6P";
        Value    0.02;
        Fixed    1;
    }
    
    Process ExpressionFluxProcess( HK )
    {
        Name    Hexokinase;
        Vmax       226.452;
        KmGLKGLCi    0.08;
        KmGLKATP     0.15;
        KeqGLK    3800.0;
        KmGLKG6P    30.0;
        KmGLKADP     0.23;
        Expression    "Vmax / (KmGLKGLCi * KmGLKATP) * (GLCi.Value * ATP.Value - G6P.Value * ADP.Value / KeqGLK) / ((1 + GLCi.Value / KmGLKGLCi + G6P.Value / KmGLKG6P) * (1 + ATP.Value / KmGLKATP + ADP.Value / KmGLKADP))";
        VariableReferenceList
            [ GLCi    Variable:/cytosol:GLCi -1 ]
            [ P       Variable:/cytosol:P    -1 ]
            [ G6P     Variable:/cytosol:G6P  1  ]
            [ ATP     Variable:/cytosol:ATP  0  ]
            [ ADP     Variable:/cytosol:ADP  0  ];
    }
    
    Process ExpressionFluxProcess( PGI )
    {
        Name    "Glucose-6-phosphate isomerase";
        Vmax        339.677;
        KmPGIG6P_2    1.4;
        KeqPGI_2      0.314;
        KmPGIF6P_2    0.3;
        Expression    "Vmax / KmPGIG6P_2 * (G6P.Value - F6P.Value / KeqPGI_2) / (1 + G6P.Value / KmPGIG6P_2 + F6P.Value / KmPGIF6P_2)";
        VariableReferenceList
            [ G6P     Variable:/cytosol:G6P  -1 ]
            [ F6P     Variable:/cytosol:F6P  1  ];
    }
    
    Process ExpressionFluxProcess( vGlycogen )
    {
        Name    "Glycogen synthesis";
        Vmax    6.0;
        Expression    "Vmax";
        VariableReferenceList
            [ G6P      Variable:/cytosol:G6P     -1 ]
            [ P        Variable:/cytosol:P       -1 ]
            [ Glycogen Variable:/cytosol:Glycogen 1 ];
    }
    
    Process ExpressionFluxProcess( vTrehalose )
    {
        Name    "Trehalose 6-phosphate synthase";
        Vmax    2.4;
        Expression    "Vmax";
        VariableReferenceList
            [ G6P       Variable:/cytosol:G6P       -2 ]
            [ P         Variable:/cytosol:P         -1 ]
            [ Trehalose Variable:/cytosol:Trehalose  1 ];
    }
    
    Process ExpressionFluxProcess( PFK )
    {
        Name    Phosphofructokinase;
        Vmax       182.903;
        gR           5.12;
        KmPFKF6P     0.1;
        KmPFKATP     0.71;
        Lzero        0.66;
        CiPFKATP   100.0;
        KiPFKATP     0.65;
        CPFKAMP      0.0845;
        KPFKAMP      0.0995;
        CPFKF26BP    0.0174;
        KPFKF26BP    0.000682;
        CPFKF16BP    0.397;
        KPFKF16BP    0.111;
        CPFKATP      3.0;
        Expression    "Vmax * gR * (F6P.Value / KmPFKF6P) * (ATP.Value / KmPFKATP) * ( 1 + F6P.Value / KmPFKF6P + ATP.Value / KmPFKATP + gR * F6P.Value / KmPFKF6P * ATP.Value / KmPFKATP ) / (pow(( 1 + F6P.Value / KmPFKF6P + ATP.Value / KmPFKATP + gR * F6P.Value / KmPFKF6P * ATP.Value / KmPFKATP ), 2) + ( Lzero * pow(( 1 + CiPFKATP * ATP.Value / KiPFKATP ) / ( 1 + ATP.Value / KiPFKATP ),2) * pow(( 1 + CPFKAMP * AMP.Value / KPFKAMP ) / ( 1 + AMP.Value / KPFKAMP ),2) * pow(( 1 + CPFKF26BP * F26BP.Value / KPFKF26BP + CPFKF16BP * F16P.Value / KPFKF16BP ) / ( 1 + F26BP.Value / KPFKF26BP + F16P.Value / KPFKF16BP ),2)) * pow(( 1.0 + CPFKATP * ATP.Value / KmPFKATP ), 2))";
        VariableReferenceList
            [ F6P       Variable:/cytosol:F6P   -1 ]
            [ P         Variable:/cytosol:P     -1 ]
            [ F16P      Variable:/cytosol:F16P  1  ]
            [ AMP       Variable:/cytosol:AMP   0  ]
            [ ATP       Variable:/cytosol:ATP   0  ]
            [ F26BP     Variable:/cytosol:F26BP 0  ];
    }
    
    Process ExpressionFluxProcess( ALD )
    {
        Name    Aldolase;
        Vmax       322.258;
        KmALDF16P    0.3;
        KeqALD       0.069;
        KmALDGAP     2.0;
        KmALDDHAP    2.4;
        KmALDGAPi   10.0;
        KeqTPI       0.045;
        Expression    "Vmax / KmALDF16P * (F16P.Value - KeqTPI / (1 + KeqTPI) * DHAP.Value * (1 / (1 + KeqTPI)) * DHAP.Value / KeqALD) / (1 + F16P.Value / KmALDF16P + KeqTPI / (1 + KeqTPI) * DHAP.Value / KmALDGAP + 1 / (1 + KeqTPI) * DHAP.Value / KmALDDHAP + KeqTPI / (1 + KeqTPI) * DHAP.Value * (1 / (1 + KeqTPI)) * DHAP.Value / (KmALDGAP * KmALDDHAP) + F16P.Value * (KeqTPI / (1 + KeqTPI)) * DHAP.Value / (KmALDGAPi * KmALDF16P))";
        VariableReferenceList
            [ F16P    Variable:/cytosol:F16P -1 ]
            [ DHAP    Variable:/cytosol:DHAP 2  ];
    }
    
    Process ExpressionFluxProcess( GAPDH )
    {
        Name    "Glyceraldehyde 3-phosphate dehydrogenase";
        Vmax_f       1184.52;
        KmGAPDHGAP      0.21;
        KmGAPDHNAD      0.09;
        Vmax_r       6549.8;
        KmGAPDHBPG      0.0098;
        KmGAPDHNADH     0.06;
        KeqTPI          0.045;
        Expression    "(Vmax_f * (KeqTPI / (1 + KeqTPI)) * DHAP.Value * NAD.Value / (KmGAPDHGAP * KmGAPDHNAD) - Vmax_r * BPG.Value * NADH.Value / (KmGAPDHBPG * KmGAPDHNADH)) / ((1 + KeqTPI / (1 + KeqTPI) * DHAP.Value / KmGAPDHGAP + BPG.Value / KmGAPDHBPG) * (1 + NAD.Value / KmGAPDHNAD + NADH.Value / KmGAPDHNADH))";
        VariableReferenceList
            [ DHAP    Variable:/cytosol:DHAP -1 ]
            [ NAD     Variable:/cytosol:NAD  -1 ]
            [ BPG     Variable:/cytosol:BPG  1  ]
            [ NADH    Variable:/cytosol:NADH 1  ];
    }
    
    Process ExpressionFluxProcess( PGK )
    {
        Name    "Phosphoglycerate kinase";
        Vmax    1306.45;
        KmPGKP3G    0.53;
        KmPGKATP    0.3;
        KeqPGK    3200.0;
        KmPGKBPG    0.003;
        KmPGKADP    0.2;
        Expression    "Vmax / (KmPGKP3G * KmPGKATP) * (KeqPGK * BPG.Value * ADP.Value - P3G.Value * ATP.Value) / ((1 + BPG.Value / KmPGKBPG + P3G.Value / KmPGKP3G) * (1 + ATP.Value / KmPGKATP + ADP.Value / KmPGKADP))";
        VariableReferenceList
            [ BPG     Variable:/cytosol:BPG  -1 ]
            [ P3G     Variable:/cytosol:P3G  1  ]
            [ P       Variable:/cytosol:P    1  ]
            [ ATP     Variable:/cytosol:ATP  0  ]
            [ ADP     Variable:/cytosol:ADP  0  ];
    }
    
    Process ExpressionFluxProcess( PGM )
    {
        Name    "Phosphoglycerate mutase";
        Vmax    2525.81;
        KmPGMP3G    1.2;
        KeqPGM    0.19;
        KmPGMP2G    0.08;
        Expression    "Vmax / KmPGMP3G * (P3G.Value - P2G.Value / KeqPGM) / (1 + P3G.Value / KmPGMP3G + P2G.Value / KmPGMP2G)";
        VariableReferenceList
            [ P3G     Variable:/cytosol:P3G  -1 ]
            [ P2G     Variable:/cytosol:P2G  1  ];
    }
    
    Process ExpressionFluxProcess( ENO )
    {
        Name    Enolase;
        Vmax    365.806;
        KmENOP2G    0.04;
        KeqENO    6.7;
        KmENOPEP    0.5;
        Expression    "Vmax / KmENOP2G * (P2G.Value - PEP.Value / KeqENO) / (1 + P2G.Value / KmENOP2G + PEP.Value / KmENOPEP)";
        VariableReferenceList
            [ P2G     Variable:/cytosol:P2G  -1 ]
            [ PEP     Variable:/cytosol:PEP  1  ];
    }
    
    Process ExpressionFluxProcess( PYK )
    {
        Name    "Pyruvate kinase";
        Vmax    1088.71;
        KmPYKPEP    0.14;
        KmPYKADP    0.53;
        KeqPYK    6500.0;
        KmPYKPYR    21.0;
        KmPYKATP    1.5;
        Expression    "Vmax / (KmPYKPEP * KmPYKADP) * (PEP.Value * ADP.Value - PYR.Value * ATP.Value / KeqPYK) / ((1 + PEP.Value / KmPYKPEP + PYR.Value / KmPYKPYR) * (1 + ATP.Value / KmPYKATP + ADP.Value / KmPYKADP))";
        VariableReferenceList
            [ PEP     Variable:/cytosol:PEP  -1 ]
            [ PYR     Variable:/cytosol:PYR  1  ]
            [ P       Variable:/cytosol:P    1  ]
            [ ATP     Variable:/cytosol:ATP  0  ]
            [ ADP     Variable:/cytosol:ADP  0  ];
    }
    
    Process ExpressionFluxProcess( PDC )
    {
        Name    "Pyruvate decarboxylase";
        Vmax    174.194;
        nPDC    1.9;
        KmPDCPYR    4.33;
        Expression    "Vmax * (pow(PYR.Value, nPDC) / pow(KmPDCPYR, nPDC)) / (1 + pow(PYR.Value, nPDC) / pow(KmPDCPYR, nPDC))";
        VariableReferenceList
            [ PYR     Variable:/cytosol:PYR  -1 ]
            [ ACE     Variable:/cytosol:ACE  1  ]
            [ CO2     Variable:/cytosol:CO2  1  ];
    }
    
    Process ExpressionFluxProcess( vSuccinate )
    {
        Name    "Succinate synthesis";
        KSUCC    21.4;
        Expression    "KSUCC * ACE.Value";
        VariableReferenceList
            [ ACE       Variable:/cytosol:ACE      -2 ]
            [ NAD       Variable:/cytosol:NAD      -3 ]
            [ P         Variable:/cytosol:P        -4 ]
            [ NADH      Variable:/cytosol:NADH      3 ]
            [ Succinate Variable:/cytosol:Succinate 1 ];
    }
    
    Process ExpressionFluxProcess( GLCtransport )
    {
        Name    "Glucose transport";
        Vmax    97.264;
        KmGLTGLCo    1.1918;
        KeqGLT    1.0;
        KmGLTGLCi    1.1918;
        Expression    "Vmax / KmGLTGLCo * (GLCo.Value - GLCi.Value / KeqGLT) / (1 + GLCo.Value / KmGLTGLCo + GLCi.Value / KmGLTGLCi + 0.91 * GLCo.Value * GLCi.Value / (KmGLTGLCo * KmGLTGLCi))";
        VariableReferenceList
            [ GLCo Variable:/:GLCo         -1 ]
            [ GLCi Variable:/cytosol:GLCi 1  ];
    }
    
    Process ExpressionFluxProcess( ADH )
    {
        Name    "Alcohol dehydrogenase";
        Vmax    810.0;
        KiADHNAD    0.92;
        KmADHETOH    17.0;
        KeqADH    6.9e-05;
        KmADHNAD    0.17;
        KmADHNADH    0.11;
        KiADHNADH    0.031;
        KmADHACE    1.11;
        KiADHACE    1.1;
        KiADHETOH    90.0;
        Expression    "-(Vmax / (KiADHNAD * KmADHETOH) * (NAD.Value * ETOH.Value - NADH.Value * ACE.Value / KeqADH) / (1 + NAD.Value / KiADHNAD + KmADHNAD * ETOH.Value / (KiADHNAD * KmADHETOH) + KmADHNADH * ACE.Value / (KiADHNADH * KmADHACE) + NADH.Value / KiADHNADH + NAD.Value * ETOH.Value / (KiADHNAD * KmADHETOH) + KmADHNADH * NAD.Value * ACE.Value / (KiADHNAD * KiADHNADH * KmADHACE) + KmADHNAD * ETOH.Value * NADH.Value / (KiADHNAD * KmADHETOH * KiADHNADH) + NADH.Value * ACE.Value / (KiADHNADH * KmADHACE) + NAD.Value * ETOH.Value * ACE.Value / (KiADHNAD * KmADHETOH * KiADHACE) + ETOH.Value * NADH.Value * ACE.Value / (KiADHETOH * KiADHNADH * KmADHACE)))";
        VariableReferenceList
            [ ACE     Variable:/cytosol:ACE    -1 ]
            [ NADH    Variable:/cytosol:NADH   -1 ]
            [ NAD     Variable:/cytosol:NAD     1 ]
            [ ETOH    Variable:/cytosol:Ethanol 1 ];
    }
    
    Process ExpressionFluxProcess( G3PDH )
    {
        Name    "Glycerol 3-phosphate dehydrogenase";
        Vmax    70.15;
        KmG3PDHDHAP    0.4;
        KmG3PDHNADH    0.023;
        KeqG3PDH    4300.0;
        KmG3PDHGLY    1.0;
        KmG3PDHNAD    0.93;
        KeqTPI    0.045;
        Expression    "Vmax / (KmG3PDHDHAP * KmG3PDHNADH) * (1 / (1 + KeqTPI) * DHAP.Value * NADH.Value - GLY.Value * NAD.Value / KeqG3PDH) / ((1 + 1 / (1 + KeqTPI) * DHAP.Value / KmG3PDHDHAP + GLY.Value / KmG3PDHGLY) * (1 + NADH.Value / KmG3PDHNADH + NAD.Value / KmG3PDHNAD))";
        VariableReferenceList
            [ DHAP    Variable:/cytosol:DHAP     -1 ]
            [ NADH    Variable:/cytosol:NADH     -1 ]
            [ NAD     Variable:/cytosol:NAD       1 ]
            [ GLY     Variable:/cytosol:Glycerol  1 ];
    }
    
    Process ExpressionFluxProcess( vATP )
    {
        Name    "ATPase activity";
        Vmax    33.7;
        Expression    "Vmax * ATP.Value";
        VariableReferenceList
            [ P       Variable:/cytosol:P    -1 ]
            [ ATP     Variable:/cytosol:ATP  0  ];
    }

Process ExpressionAssignmentProcess( Assignment_ADP )
    {
        StepperID    PSV;
        Name    "Assignment rule for 'ADP'";
        KeqAK    0.45;
        Expression    "(SUM_P.Value - pow(pow(P.Value, 2) * (1 - 4 * KeqAK) + 2 * SUM_P.Value * P.Value * (4 * KeqAK - 1) + pow(SUM_P.Value, 2), 0.5)) / (1 - 4 * KeqAK)";
        VariableReferenceList
            [ ADP   Variable:/cytosol:ADP   1 ]
            [ SUM_P Variable:/cytosol:SUM_P 0 ]
            [ P     Variable:/cytosol:P     0 ];
    }
    
    Process ExpressionAssignmentProcess( Assignment_ATP )
    {
        StepperID    PSV;
        Name    "Assignment rule for 'ATP'";
        Expression    "(P.Value - ADP.Value) / 2";
        VariableReferenceList
            [ ATP Variable:/cytosol:ATP 1 ]
            [ P   Variable:/cytosol:P   0 ]
            [ ADP Variable:/cytosol:ADP 0 ];
    }
    
    Process ExpressionAssignmentProcess( Assignment_AMP )
    {
        StepperID    PSV;
        Name    "Assignment rule for 'AMP'";
        Expression    "SUM_P.Value - ATP.Value - ADP.Value";
        VariableReferenceList
            [ AMP   Variable:/cytosol:AMP   1 ]
            [ SUM_P Variable:/cytosol:SUM_P 0 ]
            [ ATP   Variable:/cytosol:ATP   0 ]
            [ ADP   Variable:/cytosol:ADP   0 ];
    }
    
}
'''

## モデルを読み込む

In [ ]:
setModel(EM, "Teusink2000_Glycolysis")

print 't = {}'.format( getCurrentTime() )

## モデル中のすべての `Variable` と `Process` のリスト
- `System::/`：細胞外の空間（＝培養液）
- `System:/:cytosol`：細胞質
  - `Variable`：解糖系とその周辺に関係する代謝物質
  - `Process`：解糖系とその周辺に関係する酵素反応

In [ ]:
for an_E_type in ( 'Variable', 'Process' ):
    for a_SystemPath in ( '/', '/cytosol' ):
        for E in getEntityList( an_E_type, a_SystemPath ):
            print( ':'.join( ( an_E_type, a_SystemPath, E )))
        print("")

# EntityStub をつくる

- モデル中の要素（`Entity`）の内容を参照するには **Stub** という仕掛けを使います。
- 下のセルでは、`Stub_dict` という辞書に、モデル中の **すべての `Variable` と `Process`** の `Stub` を格納しています。
  - `Stub` から知ることができるのは、`Entity` の**現在**の状況です。過去の情報は含まれていません。

In [ ]:
SystemPath_list = ( '/', '/cytosol', )
Stub_dict = {}

for a_SystemPath in SystemPath_list:
    for E_type in ('Variable', 'Process'):
        for E in getEntityList( E_type, a_SystemPath ):
                FullID = ':'.join( ( E_type, a_SystemPath, E ) )
                Stub_dict[ FullID ] = createEntityStub( FullID )

def get_FullPN_value( a_FullPN ):
    a_FullPN_list = a_FullPN.split(':')
    a_FullID = ':'.join(a_FullPN_list[:3])
    return Stub_dict[ a_FullID ][ a_FullPN_list[3] ]


# Logger をつくる

- シミュレーション中に起こる、ある変数の時間変化（時系列データ）を記録する仕掛けを **Logger** といいます。
- 下のセルでは、`Logger_dict` という辞書に、モデル中の主要な変数について、その時系列データを記録するための Logger を登録しています。
- 明示的に Logger をつくらないと、シミュレーション中のデータはどんどん失われ、最新時点のデータしか保持されません。
  - すべてのデータを保持していると、大規模なモデルでは あっというまにテラバイトオーダーの容量を消費してしまうためです。
- ここでは、モデル中のすべての `Variable` の `Value` と `Velocity`、ならびにすべての `Process` の `Activity` を記録します。

In [ ]:
Target_Properties = dict(
    Variable = ['Value', 'Velocity'], 
    Process  = ['Activity'],)
Logger_dict = {}

for a_SystemPath in SystemPath_list:
    for E_type, Properties in Target_Properties.items():
        for E in getEntityList( E_type, a_SystemPath ):
            for p in Properties:
                FullPN = ':'.join( ( E_type, a_SystemPath, E, p ) )
                Logger_dict[ FullPN ] = createLoggerStub( FullPN )
                Logger_dict[ FullPN ].create()


# 初期状態を記録する
- E-Cell3は、１度のセッションで１回しかモデルを読み込めません。また、時刻も戻すことはできません。
- ただし、モデルの状態については、任意の時刻でモデル内の全状態を記録しておけば、後刻、モデル内の状態を記録済みの状態に書き換えることができます。
- 全状態とは、すべての `Variable` の `Value` です。
- 下のセルでは、辞書 `t0` に、現在（時刻 `0`）の全状態を記録しています。

In [ ]:
t0 = dict( t = getCurrentTime() )

for a_FullID, a_Stub in Stub_dict.items():
    if a_FullID.split(':')[0] == 'Variable':
        t0[ a_FullID + ':Value' ] = a_Stub['Value']

# シミュレーションを実行する

- 関数 `run( `_`n`_` )` は、時間 _n_ だけシミュレーションを実行します。このモデルの時間単位は分なので、`run( 1.0 )` は、細胞内の１分間の解糖系代謝のシミュレーションを実行します。

In [ ]:
step_width = 120.0  # min

run( step_width )
print 't = {}'.format( getCurrentTime() )

In [ ]:
print( 'GLCo.Value = {}'.format(Stub_dict['Variable:/:GLCo']['Value']))
Stub_dict['Variable:/:GLCo']['Fixed'] = 0
Stub_dict['Variable:/:GLCo']['Value'] = 100.0
Stub_dict['Variable:/:GLCo']['Fixed'] = 1
print( 'GLCo.Value = {}'.format(Stub_dict['Variable:/:GLCo']['Value']))

run( step_width )

# グラフを描くためにデータを取り出す

- Logger から時系列データを取り出して、辞書 `Data_dict` に格納する。
- `Logger.getData( 0, getCurrentTime(), getCurrentTime() / 100 )` は、**関数** `Logger.getData` に、**引数** を３つ渡しています。
  - 第１引数と第２引数は、データを取り出してくる時間帯を指定するために、最初の時刻と、最後の時刻を指定しています。
      - ここでは **時刻 `0`** から、**時刻 `getCurrentTime()`** まで、つまり、シミュレートした全時間帯のデータを取り出すように指定しています。
          - `getCurrentTime()` は、シミュレータ内の時刻を取得するための関数です。
  - 第３引数は、データの間隔を示します。今回つくった `Logger` には、記録対象についてシミュレーション中の全変化が記録されていますが、グラフを描く程度であれば、一般に 100 点くらいのデータがあれば滑らかになります。そこで、データの間隔を `getCurrentTime() / 100` と指定することで、`Logger` から、100個（厳密には101個）のデータだけをとってくるように指定しています。
      - 第３引数の刻み幅を小さくするほど、データ量は増えます。その分、処理に時間がかかるようになりますが、グラフの描画結果は正確になります。

In [ ]:
Data_dict = {}
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 0, getCurrentTime(),  getCurrentTime() / 100 ) )[ :, :2 ]

# グラフにプロットする対象をリストにする

- リスト `FullPNs_for_plot` の要素をグラフに描く。
- ここでは、解糖系の中間代謝物質のうち、G6P（グルコース６リン酸）、PYR（ピルビン酸）、および代表的なエネルギー伝達分子である ATP（アデノシン３リン酸）のモル濃度（mmol/L）をプロットする。


## グラフに描く対象を加えてみよう
- **グラフに描きたい対象の `FullPN` を下のリストに加えることができます。**
  - このモデルでは `NumberConc` がモル濃度（モル/リットル）を表します。

In [ ]:
FullPNs_for_plot = [ 
    'Variable:/cytosol:G6P:Value',
    'Variable:/cytosol:PYR:Value',
    'Variable:/cytosol:ATP:Value'
    ]

# グラフを描く（`Variable` の `Value`）

- 配列 `FullPNs_for_plot` に含まれる `FullPN` の内容をグラフに描画します。

In [ ]:
plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

# グラフを描く（`Variable` の `Velocity`）

- 上のグラフと同じ `Variable` の `Velocity` をグラフに描いてみる。

In [ ]:
FullPNs_for_plot = [ 
    'Variable:/cytosol:G6P:Velocity',
    'Variable:/cytosol:PYR:Velocity',
    'Variable:/cytosol:ATP:Velocity'
    ]

In [ ]:
plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

## ギザギザの正体を追究してみる
- 細胞外グルコースの量を２倍にした 時刻 120 分以降、３つの `Variable` の `Verocity` （物質の**変化速度**）が乱高下しています。
- 一方で、一つ前の `Value` （物質の**量**）は、120分以降も滑らかに推移しているように見えます。

- `Value` と `Verocity` の値を比べてみます。
  - 下のセルの `FullID` に任意の `Variable` の `FullID` を書き込んで実行すると、その `Value` と `Velocity` が表示されます。

In [ ]:
FullID = 'Variable:/cytosol:G6P'   # ← ここを書き換えると別のVariableの値を確認できます。 #################

print '{}\n    Value   : {}\n    Velocity: {}'.format( 
    FullID, Stub_dict[FullID]['Value'], Stub_dict[FullID]['Velocity'])

- `Value` はモル濃度、`Velocity` は１分間あたりのモル濃度の変化速度（分速）の値です。`Velocity` は、激しく振動していますが、`Value` に比べて 1000分の1 から 10000分の1 程度という、非常に小さい速度であり、`Value` に明らかな変化を与えるほどではないことがわかります。

- `Velocity`のグラフのギザギザは、かなり細かく、隣りあった点の間で大きく値が変化していることから、このグラフを描くのに使った 100 点では粗すぎるかもしれません（となりあった点の間でも大きく変化しているかもしれない）。
- そこで、1000点くらいまでデータを細かくとって、グラフを書き直してみます。
  - 具体的には `Logger.getData()` の第３引数に、より小さな値を渡します。下のセルでは、時刻 120 から 240 の区間を、0.1 分間隔でデータを取ってくるように書き直しています。

In [ ]:
Data_dict = {}
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 120, 240, 0.1 ) )[ :, :2 ]

In [ ]:
plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

- 描き直したグラフから、細胞外グルコースの量を増やした直後に大きな変化が起こり、その後も微弱な増減がつづいていることがわかります。

# モデルの数値を書き換えてみる

- モデル内の変数、パラメータを書き換えてみる。
- 書き換えたあとにシミュレーションを再開することで、その影響を計算してみる。
- 何の数値を変えるか、自分で選んで変えてみよう。
  - 変えたい物質の `FullPN` で指定します。変更する `Property` 
    - `Variable` であれば `Value`
    - `Process` であれば `Velocity`
- 現在の値を確認してから、書き換える。

## 属性を書き換えるための関数 `setEntityProperty()` を用意する。
- 書式：`setEntityProperty( `*`FullPN`*`, `*`value`*` )`
  - 引数
    - *`FullPN`* ：書き換えたい `Property` の `FullPN`
    - *`value`* ：書き換える値

In [ ]:
def setEntityProperty( a_FullPN, a_value ):
    a_FullPN_list = a_FullPN.split(':')
    a_FullID = ':'.join(a_FullPN_list[:3])
    a_Property = a_FullPN_list[3]
    if a_FullPN_list[0] == 'Variable':
        if Stub_dict[a_FullID]['Fixed'] == 1:
            Stub_dict[a_FullID]['Fixed'] = 0
            Stub_dict[a_FullID][ a_FullPN_list[3] ] = a_value
            Stub_dict[a_FullID]['Fixed'] = 1
        else:
            Stub_dict[a_FullID][ a_FullPN_list[3] ] = a_value
    elif a_FullPN_list[0] == 'Process':
        Stub_dict[a_FullID][ a_FullPN_list[3] ] = a_value

    print( '{} = {}'.format( a_FullPN, Stub_dict[a_FullID][ a_FullPN_list[3] ] ) )
    return

## 関数 `setEntityProperty()` を使って、属性を書き換える
- 下のセルの例では、ピルビン酸（`Variable:/cytosol:PYR:Value`）を 50.0 M に変更している。
- セルを実行すると、書き換え後の値が `print` されます。

In [ ]:
taeget_FullPN = 'Variable:/cytosol:PYR:Value'  # ←ここを書き換えたい FullPN に書き換える ##############

setEntityProperty( taeget_FullPN, 50.0 )       # ←第２引数に書き換えたい値を設定する ##############

# シミュレーションしてグラフを描く

- 書き換え後にシミュレーションを行い、その結果をグラフに描く。
- ほぼコピペでＯＫ。

In [ ]:
run( step_width )
print 't = {}'.format( getCurrentTime() )

- Logger には自動的にデータが貯まりますが、`Data_dict` は手動で更新しなければなりません。
- `getCurrentTime()` を使って、これまでのシミュレーションしたデータから、最初から最後までを、100区画に刻んだデータを取ってきます。

In [ ]:
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 0, getCurrentTime(), getCurrentTime() / 100 ) )[ :, :2 ]

plt.figure()
plt.plot( Data_dict[ taeget_FullPN ][ :, 0 ], Data_dict[ taeget_FullPN ][ :, 1 ], label = taeget_FullPN.split(':')[ 2 ] )
plt.legend( loc = 'upper right' )

# 他の `Variable` もグラフに描いてみる

- 配列 `FullPNs_for_plot` に含まれる `FullPN` の内容をグラフに描画します。

In [ ]:
FullPNs_for_plot = [ 
    'Variable:/cytosol:G6P:Value',
    'Variable:/cytosol:PYR:Value',
    'Variable:/cytosol:ATP:Value'
    ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

# モデルを時刻０の状態にリセットする
- モデルを読み込んだ際に、全状態を記録した辞書 `t0` を利用して、モデルの状態を時刻０の状態に戻す。

In [ ]:
for a_FullPN, a_value in t0.items():
    if a_FullPN != 't':
        setEntityProperty( a_FullPN, a_value )

# シミュレーションしてリセットされたことを確認する

In [ ]:
run( step_width )

for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 0, getCurrentTime(), getCurrentTime() / 100 ) )[ :, :2 ]

FullPNs_for_plot = [ 
    'Variable:/cytosol:G6P:Value',
    'Variable:/cytosol:ATP:Value'
    ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

- グラフを見るかぎり、リセットできている様子。

# 辺縁（ボーダー）にある要素の状況を確認する
- 物質量（`Value`）をグラフ化してみる。

In [ ]:
FullPNs_for_plot = [ 
    'Variable:/:GLCo:Value',
    'Variable:/cytosol:Glycogen:Value',
    'Variable:/cytosol:Trehalose:Value',
    'Variable:/cytosol:Glycerol:Value',
    'Variable:/cytosol:Succinate:Value',
    'Variable:/cytosol:Ethanol:Value',
    ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

- 細胞外グルコースは２倍に書き換えたために、矩形状に増減しているが、他の物質は、**固定されているため**一切変化していない。

## 辺縁にある要素の変化速度を確認する
- 変化速度（`Velocity`）をグラフ化してみる。
- 辺縁にある物質は、下のセルにある６つ。

In [ ]:
FullPNs_for_plot = [ 
    'Variable:/:GLCo:Velocity',
    'Variable:/cytosol:Glycogen:Velocity',
    'Variable:/cytosol:Trehalose:Velocity',
    'Variable:/cytosol:Glycerol:Velocity',
    'Variable:/cytosol:Succinate:Velocity',
    'Variable:/cytosol:Ethanol:Velocity',
    ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

- 物質の変化速度（`Velocity`）は、細胞外グルコースの量の変化の影響を受けている。

# 演習
- モデル中のさまざまな要素の属性の値を書き換えて、その結果を観察してみよう。
- 属性の値を書き換えた結果は、いくつかに分類できます。どのような変化の種類があり、それらを分けている原因は何なのか、考えてみましょう。